In [6]:
#dependencies
import re
import numpy as np
import pandas as pd
import csv
from pprint import pprint
import math

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from nltk.tokenize import word_tokenize



# Plotting tools
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
#load data
#load training Data
with open('../../../data/Datasets/20subs_long/idocnade/training.csv', newline='') as csvfile1:
    data_train = pd.read_csv(csvfile1,  names=["Label", "Text"])
    train_fold = data_train.dropna(subset=['Text'])   

#load validation Data
with open('../../../data/Datasets/20subs_long/idocnade/validation.csv', newline='') as csvfile2:
    data_val = pd.read_csv(csvfile2,  names=["Label", "Text"])
    val_fold = data_val.dropna(subset=['Text'])     
    
frames = [train_fold,val_fold]
train_data = pd.concat(frames)


#load testing Data
with open('../../../data/Datasets/20subs_long/idocnade/test.csv', newline='') as csvfile3:
    data_test = pd.read_csv(csvfile3,  names=["Label", "Text"])  
    test_data = data_test.dropna(subset=['Text'])  

In [8]:
#perform cross-validation for alpha, beta, k
hyperparameter_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

alpha = [0.1,0.3,0.5,0.7,1]
beta = [0.1,0.3,0.5,0.7,1]


#data prep
dictionary = gensim.corpora.Dictionary(train_fold["Text"].str.split())
corpus_train = [dictionary.doc2bow(text.split()) for index,text in train_fold["Text"].iteritems()]
corpus_val = [dictionary.doc2bow(text.split()) for index,text in val_fold["Text"].iteritems()]
texts = [[dictionary[word_id] for word_id, freq in doc] for doc in corpus_val]

for k in tqdm(range(1, 200, 3)):
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    lda_model = gensim.models.LdaMulticore(corpus=corpus_train,
                                                           id2word = dictionary, 
                                                           num_topics = k, 
                                                           alpha=a,
                                                           eta=b,
                                                           workers=15)
                    # Coherence
                    coherence_model_small = gensim.models.coherencemodel.CoherenceModel(model=lda_model,
                                                                                        topn = 10,
                                                                                        texts = texts,
                                                                                        coherence='c_v')
                    coherence_model_large = gensim.models.coherencemodel.CoherenceModel(model=lda_model,
                                                                                        topn = 20,
                                                                                        texts = texts,
                                                                                        coherence='c_v')
                    coherence1 = coherence_model_small.get_coherence()
                    coherence2 = coherence_model_large.get_coherence()
                    c_v = (coherence1+coherence2)/2
                    hyperparameter_results['Topics'].append(k)
                    hyperparameter_results['Alpha'].append(a)
                    hyperparameter_results['Beta'].append(b)
                    hyperparameter_results['Coherence'].append(c_v)

/home/flokla/anaconda3/lib/python3.8/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/home/flokla/anaconda3/lib/python3.8/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


In [10]:
with open('../../../LDA/model/20subs_long/hyperparameter_tuning.', 'w') as f:
    print(hyperparameter_results, file=f)


In [11]:
#best hyperparameter
a = 0.1
b = 1
k = 43

In [12]:
#save best lda model with optimal alpha beta for k=*
dictionary = gensim.corpora.Dictionary(train_data["Text"].str.split())
corpus_train = [dictionary.doc2bow(text.split()) for index,text in train_data["Text"].iteritems()]
lda_model_k_star = gensim.models.LdaMulticore(corpus=corpus_train,
                                                           id2word = dictionary, 
                                                           num_topics = k, 
                                                           alpha=a,
                                                           eta=b,
                                                           workers=15)
lda_model_k_star.save('../../../LDA/model/20subs_long/lda.model_with_optimal_k/lda')

In [13]:
#save best lda model with optimal alpha beta for k=20
lda_model_k_20 = gensim.models.LdaMulticore(corpus=corpus_train,
                                                           id2word = dictionary, 
                                                           num_topics = 20, 
                                                           alpha=a,
                                                           eta=b,
                                                           workers=15)
lda_model_k_20.save('../../../LDA/model/20subs_long/lda.model_with_k20/lda')

In [14]:
#save best lda model with optimal alpha beta for k=50
lda_model_k_50 = gensim.models.LdaMulticore(corpus=corpus_train,
                                                           id2word = dictionary, 
                                                           num_topics = 50, 
                                                           alpha=a,
                                                           eta=b,
                                                           workers=15)
lda_model_k_50.save('../../../LDA/model/20subs_long/lda.model_with_k50/lda')

In [15]:
#save best lda model with optimal alpha beta for k=100
lda_model_k_100 = gensim.models.LdaMulticore(corpus=corpus_train,
                                                           id2word = dictionary, 
                                                           num_topics = 100, 
                                                           alpha=a,
                                                           eta=b,
                                                           workers=15)
lda_model_k_100.save('../../../LDA/model/20subs_long/lda.model_with_k100/lda')

In [16]:
#save best lda model with optimal alpha beta for k=200
lda_model_k_200 = gensim.models.LdaMulticore(corpus=corpus_train,
                                                           id2word = dictionary, 
                                                           num_topics = 200, 
                                                           alpha=a,
                                                           eta=b,
                                                           workers=15)
lda_model_k_200.save('../../../LDA/model/20subs_long/lda.model_with_k200/lda')

In [9]:
#perform evaluation

In [10]:
#perplexity

In [11]:
#coherence for top10 and top20 topics words per topic, average across all topics, save highest and lowest c_v

In [12]:
#perform IR

In [13]:
#perform logistisc regression

In [14]:
#print some topics


0       d need stay probably negative voltage supply o...
1       confused developer work grant web designer nee...
2            d like express mod team allow post thank lot
3       fun thing loose capacity function level requir...
4                                               thank try
                              ...                        
9435                  read require tuner min subscription
9436    pound powder primer brass medical bill dollar ...
9437                                       oppose vehicle
9438    feed beast come malady live stop drag decent b...
9439                             work right hold consider
Name: Text, Length: 9351, dtype: object